### Mouse Protein dataset regression (accuracy)

The 'Data_Cortex_Nuclear.csv' dataset is from the **abidlabs/contrastive** Github repository: https://github.com/abidlabs/contrastive/blob/master/experiments/datasets/Data_Cortex_Nuclear.csv

In [ ]:
%pip install contrastive-inverse-regression

In [4]:
import numpy as np
import pandas as pd
import time
import scipy
from scipy.linalg import eigh
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
import contrastive_inverse_regression
from contrastive_inverse_regression import CIR

data = pd.read_csv('datasets/Data_Cortex_Nuclear.csv')

# foreground data
fg = data.dropna()

# foreground label
Y = fg['class']
Y = pd.Categorical(Y)
Y = Y.rename_categories({'c-CS-m': '0', 'c-CS-s': '1', 'c-SC-m': '2', 'c-SC-s': '3',
                         't-CS-m': '4', 't-CS-s': '5', 't-SC-m': '6', 't-SC-s': '7'})
Y = Y.astype(float)
labels = np.unique(Y)      # set of unique foreground labels

# foreground slices
L = len(labels)         # number of foreground classes/slices
fg = fg.iloc[:, 1:78]
n = fg.shape[0]  # foreground sample size
X = fg - np.mean(fg, axis=0)
X = X.values

# background data
bg = data[data['Genotype'] == 'Control'].copy()
bg = bg.dropna()

# background label
Yt = bg['Behavior']
Yt = pd.Categorical(Yt)
Yt = Yt.rename_categories({'C/S': '0', 'S/C': '1'})
Yt = Yt.astype(float)
labelst = np.unique(Yt)   # set of unique background labels
Lt = len(labelst)       # number of background classes/slices
bg = bg.iloc[:, 1:78]
m, p = bg.shape

# tuning parameter
alpha = 0.0002
d = 2

Ds = list(range(2, 8))
alpha = 0.0002 


Accuracy_Tree = []
Accuracy_KNN = []
Accuracy_SVM = []
Accuracy_boosting = []
Accuracy_NN = []

Accuracy_KNN_std = []
Accuracy_Tree_std = []
Accuracy_SVM_std = []
Accuracy_boosting_std = []
Accuracy_NN_std = []

for d in Ds:

    # PCA
    print('PCA......')
    pca = PCA(n_components=d)
    X_PCA = pca.fit_transform(X)

    # CPCA
    print('CPCA......')
    cov_fg = np.cov(fg, rowvar=False)
    cov_bg = np.cov(bg, rowvar=False)
    eigvals, eigvecs = np.linalg.eig(cov_fg - alpha * cov_bg)
    idx = eigvals.argsort()[::-1][:d]
    V_CPCA = eigvecs[:, idx]
    X_CPCA = X @ V_CPCA

    # LDA
    print('LDA......')
    lda = LDA(n_components=d)
    X_LDA = lda.fit_transform(X, Y)


    # LASSO
    print('LASSO......')
    lasso = LassoCV().fit(X, Y)
    B = lasso.coef_
    selected_idx = np.argsort(np.abs(B))[-d:]
    X_LASSO = X[:, selected_idx]

    # SIR
    print('SIR......')
    Sigma_XX = X.T @ X / n
    Sigma_X = np.zeros((p, p))

    for l in range(1, L):
        X_curr = fg.values[Y == l]
        n_curr = X_curr.shape[0]
        Sigma_X += n_curr * np.outer(X_curr.mean(axis=0) - fg.values.mean(axis=0), X_curr.mean(axis=0) - fg.values.mean(axis=0))


    Sigma_X /= n
    eigvals, eigvecs = scipy.linalg.eig(Sigma_XX, Sigma_X)
    V_SIR = eigvecs[:, :d]
    X_SIR = X @ V_SIR


    # CIR
    print('CIR......')
    # Assuming CIR is a defined function
    V_CIR = CIR(fg, Y, bg, Yt, alpha, d)
    X_CIR = X @ V_CIR



    T = 10
    cross_val_KNN = []
    cross_val_Tree = []
    cross_val_SVM = []
    cross_val_Boosting = []
    cross_val_NN = []


    for t in range(T):                      
        # print(f't = {t}')

        # KNN
        print('Training KNN')
        knn = KNeighborsClassifier()

        cross_val_KNN.append([
            cross_val_score(knn, fg, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_PCA, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_CPCA, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_LDA, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_LASSO, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_SIR, Y, cv=10, scoring='accuracy').mean(),
            cross_val_score(knn, X_CIR, Y, cv=10, scoring='accuracy').mean()
        ])

        # Decision Tree
        print('Training Decision Tree')
        tree = DecisionTreeClassifier()
        cross_val_Tree.append([
            cross_val_score(tree, fg, Y, cv=10).mean(),
            cross_val_score(tree, X_PCA, Y, cv=10).mean(),
            cross_val_score(tree, X_CPCA, Y, cv=10).mean(),
            cross_val_score(tree, X_LDA, Y, cv=10).mean(),
            cross_val_score(tree, X_LASSO, Y, cv=10).mean(),
            cross_val_score(tree, X_SIR, Y, cv=10).mean(),
            cross_val_score(tree, X_CIR, Y, cv=10).mean()
        ])


        # SVM
        print('Training SVM')
        svm = SVC() 
        cross_val_SVM.append([
            cross_val_score(svm, fg, Y, cv=10).mean(),
            cross_val_score(svm, X_PCA, Y, cv=10).mean(),
            cross_val_score(svm, X_CPCA, Y, cv=10).mean(),
            cross_val_score(svm, X_LDA, Y, cv=10).mean(),
            cross_val_score(svm, X_LASSO, Y, cv=10).mean(),
            cross_val_score(svm, X_SIR, Y, cv=10).mean(),
            cross_val_score(svm, X_CIR, Y, cv=10).mean()
        ])



        # Boosting
        print('Training Boosting')
        boosting = AdaBoostClassifier()
        cross_val_Boosting.append([
            cross_val_score(boosting, fg, Y, cv=10).mean(),
            cross_val_score(boosting, X_PCA, Y, cv=10).mean(),
            cross_val_score(boosting, X_CPCA, Y, cv=10).mean(),
            cross_val_score(boosting, X_LDA, Y, cv=10).mean(),
            cross_val_score(boosting, X_LASSO, Y, cv=10).mean(),
            cross_val_score(boosting, X_SIR, Y, cv=10).mean(),
            cross_val_score(boosting, X_CIR, Y, cv=10).mean()
        ])


        # Neural Network
        print('Training Neural Network')
        nn = MLPClassifier()
        cross_val_NN.append([
            cross_val_score(nn, fg, Y, cv=10).mean(),
            cross_val_score(nn, X_PCA, Y, cv=10).mean(),
            cross_val_score(nn, X_CPCA, Y, cv=10).mean(),
            cross_val_score(nn, X_LDA, Y, cv=10).mean(),
            cross_val_score(nn, X_LASSO, Y, cv=10).mean(),
            cross_val_score(nn, X_SIR, Y, cv=10).mean(),
            cross_val_score(nn, X_CIR, Y, cv=10).mean()
        ])


    KNN_mean = np.mean(cross_val_KNN, axis=0)
    KNN_std = np.std(cross_val_KNN, axis=0)
    Accuracy_KNN.append(KNN_mean)
    Accuracy_KNN_std.append(KNN_std)

    Tree_mean = np.mean(cross_val_Tree, axis=0)
    Tree_std = np.std(cross_val_Tree, axis=0)
    Accuracy_Tree.append(Tree_mean)
    Accuracy_Tree_std.append(Tree_std)

    SVM_mean = np.mean(cross_val_SVM, axis=0)
    SVM_std = np.std(cross_val_SVM, axis=0)
    Accuracy_SVM.append(SVM_mean)
    Accuracy_SVM_std.append(SVM_std)
    
    Boosting_mean = np.mean(cross_val_Boosting, axis=0)
    Boosting_std = np.std(cross_val_Boosting, axis=0)
    Accuracy_SVM.append(Boosting_mean)
    Accuracy_SVM_std.append(Boosting_std)

    NN_mean = np.mean(cross_val_NN, axis=0)
    NN_std = np.std(cross_val_NN, axis=0)
    Accuracy_SVM.append(NN_mean)
    Accuracy_SVM_std.append(NN_std)


PCA......
CPCA......
LDA......
LASSO......
SIR......
CIR......


/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


---------------------------------------------------

Results for Scaled Gradient Projection Method 

---------------------------------------------------

   Obj. function = -1.205786e+00

   Gradient norm = 1.748532e-04 

   ||X^T*X-I||_F = 4.80e-16

   Iteration number = 3000

   Cpu time (secs) = 4.2277

   Number of evaluation(Obj. func) = 3338

Training KNN
Error occurred: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py", line 238, in fit
    return self._fit(X, y)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/skle

In [ ]:

# # Plotting KNN Accuracy
# plt.figure(figsize=(10, 6))
# for j in range(Accuracy_KNN.shape[1]):
#     plt.plot(Ds[:6], Accuracy_KNN[:6, j], label=f'Classifier {j+1}', linewidth=2)

# plt.legend(fontsize=12)
# plt.title('KNN Accuracy', fontsize=16)
# plt.xlabel('Ds', fontsize=14)
# plt.ylabel('Accuracy', fontsize=14)
# plt.grid(True)
# plt.show() 



# # Plotting KNN Accuracy
# plt.figure(figsize=(10, 6))
# for j in range(Accuracy_KNN_mean.shape[1]):
#     plt.plot(Ds[:6], Accuracy_KNN_mean[:6, j], label=f'Classifier {j+1}', linewidth=2)

# plt.legend(fontsize=12)
# plt.title('KNN Accuracy', fontsize=16)
# plt.xlabel('Ds', fontsize=14)
# plt.ylabel('Accuracy', fontsize=14)
# plt.grid(True)
# plt.show()

# # Example of matrix to LaTeX conversion (adjust as needed)
# def matrix2latex(matrix, filename):
#     # Function to convert matrix to LaTeX format
#     with open(filename + '.tex', 'w') as f:
#         for row in matrix:
#             f.write(' & '.join([f'{val:.2f}' for val in row]) + r' \\ \hline' + '\n')

# matrix2latex(Accuracy_KNN_mean.T, 'Mouse_Accuracy_KNN')
# matrix2latex(Accuracy_Tree_mean.T, 'Mouse_Accuracy_Tree')
# matrix2latex(Accuracy_SVM_mean.T, 'Mouse_Accuracy_SVM')
# matrix2latex(Accuracy_boosting_mean.T, 'Mouse_Accuracy_Boosting')
# matrix2latex(Accuracy_NN_mean.T, 'Mouse_Accuracy_NN')

# matrix2latex(Accuracy_KNN_std.T, 'Mouse_Accuracy_std_KNN')
# matrix2latex(Accuracy_Tree_std.T, 'Mouse_Accuracy_std_Tree')
# matrix2latex(Accuracy_SVM_std.T, 'Mouse_Accuracy_std_SVM')
# matrix2latex(Accuracy_boosting_std.T, 'Mouse_Accuracy_std_Boosting')
# matrix2latex(Accuracy_NN_std.T, 'Mouse_Accuracy_std_NN')

In [ ]:



print("CIR......")
accurary_group = []


for i in range(1):
    V_CIR = CIR(fg, Y, bg, Yt, alpha, d)
    X_CIR = X @ V_CIR
    mdl_CIR_KNN = KNeighborsClassifier(n_neighbors=1)
    # kf = KFold(n_splits=10)
    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_results = cross_val_score(
        mdl_CIR_KNN, X_CIR, Y, cv=10, scoring='accuracy')
    # print(cross_val_results)
    Accuracy_CIR_KNN = np.mean(cross_val_results)
    accurary_group.append(Accuracy_CIR_KNN)

print("this is CIR_KNN ", accurary_group)
print("This is mean ", np.mean(accurary_group))
print("This is standard deviation", np.std(accurary_group))


accurary_group_tree = []

for i in range(1):
    V_CIR = CIR(fg, Y, bg, Yt, alpha, d)
    X_CIR = X @ V_CIR
    mdl_CIR_Tree = DecisionTreeClassifier()
    # kf = KFold(n_splits=10)
    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_results = cross_val_score(
        mdl_CIR_Tree, X_CIR, Y, cv=10, scoring='accuracy')
    # print(cross_val_results)
    Accuracy_CIR_Tree = cross_val_results.mean()
    accurary_group_tree.append(Accuracy_CIR_Tree)

print("this is CIR_Tree ", accurary_group_tree)
print("This is mean ", np.mean(accurary_group_tree))
print("This is standard deviation ", np.std(accurary_group_tree))





